In [ ]:
pip install -q yahoo_fin

In [1]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [7]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


Mounted at /content/drive
/content/drive/MyDrive/colab/results/gstock-11a-adjclose-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
/content/drive/MyDrive/colab/results/gstock-11a-high-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
/content/drive/MyDrive/colab/results/gstock-11a-low-sh-1-sc-minmax-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras


In [8]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

pd.options.display.max_columns = None

In [ ]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DLO', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '12b', tickers, False)

In [ ]:
addAlloc(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
31,GLBE,1.05,0.96,490.40,402.54,56.78,65.72,0.16,0.63,0.70,418.36,363.30,63.57,0.12,0.63,0.71,504.05,389.16,53.76,-0.05,9.350000
46,MNDY,6.80,0.94,2030.02,2304.14,248.13,267.59,0.08,4.21,0.72,2559.06,2171.34,256.49,0.03,4.20,0.75,2773.69,2365.31,236.97,-0.04,8.650000
10,CFLT,1.26,0.89,294.45,612.00,28.74,37.38,0.30,0.78,0.72,441.79,627.18,39.03,0.36,0.66,0.86,535.76,696.49,26.54,-0.08,8.533333
5,BASE,0.53,0.91,217.18,305.79,17.29,19.73,0.14,0.49,0.67,166.79,185.43,19.54,0.13,0.38,0.80,261.04,306.11,15.51,-0.10,8.457143
29,GFS,1.13,0.91,609.22,596.68,41.67,46.90,0.13,0.68,0.69,529.50,566.62,48.93,0.17,0.56,0.74,606.95,605.89,41.48,-0.00,8.407692
83,U,5.18,0.86,410.90,965.21,22.32,34.68,0.55,5.21,0.73,293.25,780.91,37.73,0.69,3.60,0.72,365.82,776.24,22.77,0.02,8.345455
7,BMBL,0.78,0.89,158.20,287.74,7.50,8.42,0.12,0.59,0.62,115.63,191.36,9.49,0.27,0.59,0.68,185.26,239.58,7.15,-0.05,7.983333
87,WOLF,6.42,0.80,467.10,979.98,6.67,9.99,0.50,5.03,0.66,401.71,997.92,10.96,0.64,4.79,0.69,323.16,943.77,6.17,-0.07,7.600000
13,CVNA,12.87,0.88,1931.44,1303.39,240.47,254.33,0.06,9.76,0.72,1785.70,1374.67,271.94,0.13,9.91,0.70,1245.74,797.71,221.34,-0.08,6.800000
44,MBLY,1.07,0.89,198.44,341.92,16.54,17.35,0.05,0.53,0.85,337.63,412.25,20.09,0.21,0.65,0.67,142.98,297.69,15.33,-0.07,6.700000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
49,XME,0.68,0.97,392.56,371.59,60.69,55.83,-0.08,0.44,0.79,350.45,315.67,60.42,-0.00,0.55,0.64,363.95,259.80,53.26,-0.12,9.325000
4,DAPP,0.34,0.96,148.18,182.13,16.87,18.75,0.11,0.26,0.71,158.97,145.53,20.63,0.22,0.25,0.66,130.40,107.76,15.50,-0.08,9.236364
7,DXYZ,2.07,0.93,378.84,74.59,62.91,53.21,-0.15,1.87,0.82,350.60,39.15,62.84,-0.00,1.09,0.55,95.70,-318.62,52.98,-0.16,8.833333
6,DTEC,0.35,0.96,209.30,173.01,47.70,49.59,0.04,0.24,0.71,150.19,153.57,48.12,0.01,0.24,0.64,181.06,135.37,45.94,-0.04,8.600000
20,JETS,0.31,0.96,190.02,154.07,26.25,27.24,0.04,0.16,0.78,145.19,145.35,28.84,0.10,0.21,0.66,102.83,77.59,25.30,-0.04,8.600000
40,XLE,0.98,0.95,605.00,432.48,91.31,87.06,-0.05,0.63,0.83,568.18,413.59,92.96,0.02,0.76,0.68,497.27,264.52,85.88,-0.06,8.500000
41,XLF,0.30,0.96,187.98,158.93,50.85,49.49,-0.03,0.21,0.80,179.19,110.36,49.55,-0.03,0.24,0.63,148.60,80.13,45.98,-0.10,8.266667
48,XLY,1.52,0.95,1148.04,855.93,230.01,220.52,-0.04,1.10,0.73,828.58,645.42,230.46,0.00,1.15,0.66,1017.61,701.53,215.19,-0.06,8.250000
24,MTUM,1.52,0.95,913.42,624.58,223.01,214.26,-0.04,0.80,0.82,805.63,523.72,216.08,-0.03,0.89,0.63,620.82,458.37,203.46,-0.09,8.250000
9,FDN,2.02,0.97,1273.08,791.16,258.50,263.32,0.02,1.37,0.72,993.77,611.73,258.72,0.00,1.41,0.67,1184.25,831.61,245.79,-0.05,8.200000


In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
6,APPS,3.22,0.89,119.07,389.05,2.50,3.22,0.29,3.22,0.62,5.41,273.15,3.30,0.32,1.17,0.68,37.48,364.03,2.18,-0.13,4.855984
37,LUMN,0.57,0.88,46.05,99.02,4.93,6.69,0.36,0.53,0.68,64.04,87.76,6.40,0.30,0.38,0.70,23.08,29.23,4.99,0.01,4.843137
59,SLDP,0.20,0.86,46.25,79.13,1.40,1.92,0.38,0.19,0.60,8.97,43.09,1.83,0.31,0.10,0.73,31.83,68.33,1.37,-0.02,4.690402
46,OPEN,1.51,0.85,52.11,99.35,1.44,2.06,0.43,1.50,0.64,25.98,73.04,2.20,0.53,0.80,0.76,35.35,100.56,1.66,0.16,4.651163
20,DLO,0.76,0.85,153.81,285.68,13.56,19.03,0.40,1.00,0.70,112.43,268.39,22.45,0.66,0.46,0.64,139.60,290.98,12.41,-0.09,4.625000
62,SST,0.33,0.82,60.72,121.44,0.68,1.15,0.69,0.49,0.50,163.68,210.41,1.10,0.62,0.22,0.68,33.06,101.75,0.79,0.16,4.562660
56,RVYL,6.63,0.76,115.83,375.80,1.23,4.90,2.98,6.13,0.54,139.52,278.82,3.67,1.98,1.85,0.66,62.09,232.28,1.84,0.49,4.390051
63,STEM,1.52,0.76,56.41,132.35,0.60,1.89,2.15,1.75,0.56,15.38,86.05,1.20,1.00,0.88,0.65,10.19,68.06,0.56,-0.06,4.358960
11,BKKT,22.91,0.76,447.14,1272.55,18.48,40.71,1.20,38.74,0.47,220.12,994.89,37.53,1.03,7.76,0.74,299.03,1494.75,15.87,-0.14,4.270588
2,ALSMY,0.27,0.88,23.65,23.93,1.96,2.22,0.13,0.29,0.68,10.33,14.72,2.10,0.07,0.24,0.62,9.55,13.06,1.88,-0.04,4.253394


In [ ]:
# read csv file
pred_other = pd.read_csv('results/pennystock_predictions_12b.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other = pred_other.merge(df3[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')



In [ ]:
pred_other.head()

,ticker,current_price,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain
0,AIRI,4.44,2025-01-24,4.09,4.84,4.66,AIRI,3.65,5.58,3.17,-0.18
1,AEXAF,0.00,2025-01-24,0.00,0.00,0.00,AEXAF,0.90,0.59,0.12,299.97
2,ALSMY,2.22,2025-01-24,2.14,2.32,2.40,ALSMY,2.06,2.44,2.13,-0.07
3,AMSC,25.06,2025-01-24,18.64,25.48,20.27,AMSC,18.58,30.70,15.83,-0.26
4,AMSSY,3.54,2025-01-24,3.06,5.00,3.79,AMSSY,4.44,4.58,2.05,0.26


In [ ]:
# Get all entries where predicted_close > current_price and Gain > 0

result = pred_other[(pred_other['predicted_close'] > pred_other['current_price']) & (pred_other['Gain'] > 0)]
# And largest based on 'Alloc
result = result.sort_values('Alloc', ascending=False)
result


,ticker,current_price,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
18,CRCT,5.75,2025-01-24,4.63,6.14,6.13,CRCT,7.62,6.88,5.00,0.32,5.071691
24,GOGO,7.69,2025-01-24,6.78,7.75,7.91,GOGO,8.41,8.68,7.34,0.09,5.032680
4,AMSSY,3.54,2025-01-24,3.06,5.00,3.79,AMSSY,4.44,4.58,2.05,0.26,4.812217
46,OPEN,1.67,2025-01-24,1.24,1.80,1.77,OPEN,2.20,2.24,1.55,0.32,4.801471
58,SAVE,0.40,2025-01-24,0.42,0.59,0.56,SAVE,1.42,1.29,0.07,2.56,4.753217
51,PLUG,2.38,2025-01-24,2.25,3.05,2.65,PLUG,4.34,2.93,1.99,0.82,4.746055
44,OMQS,0.17,2025-01-24,0.15,0.26,0.24,OMQS,0.39,0.44,0.15,1.24,4.745256
25,GRAB,4.81,2025-01-24,4.59,5.02,4.92,GRAB,5.44,7.41,3.95,0.13,4.660633
47,OTLY,0.64,2025-01-24,0.57,0.66,0.66,OTLY,1.15,1.04,0.51,0.80,4.455882
64,STEM,0.60,2025-01-24,0.51,0.65,0.62,STEM,1.75,1.03,0.50,1.94,4.410855


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
26,META,8.02,0.97,3794.63,1664.59,688.37,610.90,-0.11,6.00,0.78,3836.22,1381.18,688.48,0.00,5.13,0.70,2773.50,735.38,498.78,-0.28,6.655844
4,AMZN,3.10,0.96,1671.51,883.28,233.77,205.71,-0.12,1.74,0.82,1425.03,913.93,234.16,0.00,1.49,0.71,1335.32,1086.15,190.53,-0.18,6.523810
1,ACN,4.67,0.93,2125.91,1720.03,384.83,303.90,-0.21,3.26,0.79,2012.36,1404.43,378.51,-0.02,2.75,0.72,1896.34,1541.58,336.63,-0.13,6.309524
12,COST,11.90,0.91,5137.56,2227.70,978.47,727.57,-0.26,6.99,0.80,4488.53,1642.54,954.29,-0.02,5.77,0.69,4083.76,683.85,827.48,-0.15,6.153846
20,HD,5.05,0.91,2270.81,1404.48,416.38,334.69,-0.20,3.09,0.79,2464.20,1616.96,414.30,-0.01,2.80,0.60,1221.54,429.30,352.79,-0.15,6.050000
18,GOOGL,2.28,0.90,921.36,641.00,199.74,153.12,-0.23,1.80,0.77,1101.88,629.13,198.59,-0.01,1.37,0.62,724.18,480.96,171.00,-0.14,5.993789
19,FDX,4.28,0.92,1677.91,1512.85,265.64,231.55,-0.13,2.98,0.77,2105.18,1793.72,278.14,0.05,2.41,0.72,1902.69,1304.46,229.04,-0.14,5.956044
40,UPS,2.70,0.90,698.42,981.62,111.34,131.96,0.19,1.74,0.76,674.44,1193.59,130.24,0.17,1.58,0.70,668.95,1152.04,121.71,0.09,5.902256
7,BKNG,57.27,0.91,28747.38,14212.10,4758.55,4074.35,-0.14,38.87,0.80,27620.39,14234.91,4895.22,0.03,35.58,0.67,19939.31,7458.83,4108.20,-0.14,5.857143
22,IBM,2.67,0.90,1099.68,625.37,259.31,215.64,-0.17,1.85,0.74,1169.17,445.25,240.17,-0.07,1.56,0.66,914.26,123.03,198.11,-0.24,5.840336


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '1a', tickers5, True)

In [ ]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
61,THQQF,1.15,0.62,1.99,5.95,2.75,3.45,0.25,1.52,0.43,-3.93,-0.03,4.28,0.55,0.85,0.54,0.89,1.72,2.93,0.06,1.388571
1,ADDDF,17.99,0.50,24.36,66.20,239.00,251.01,0.05,10.75,0.53,-46.01,-29.26,263.10,0.10,10.92,0.59,24.82,75.86,235.07,-0.02,0.000000
2,ADYEY,1.80,0.53,6.24,-6.80,14.78,15.40,0.04,1.15,0.36,-19.18,-4.64,16.87,0.14,1.06,0.58,9.10,4.96,14.27,-0.03,0.000000
3,BAK,1.24,0.33,-10.30,-1.91,3.78,4.45,0.18,1.07,0.43,-9.15,2.92,5.14,0.36,0.82,0.59,-4.37,12.86,4.00,0.06,0.000000
0,ASML,29.28,0.68,-85.94,522.02,714.36,726.59,0.02,25.88,0.51,43.25,143.26,790.66,0.11,18.20,0.62,561.13,560.81,663.80,-0.07,0.000000
5,BESIY,6.67,0.43,42.67,-52.17,137.69,142.03,0.03,4.66,0.33,-114.71,15.85,151.01,0.10,3.91,0.50,-5.09,-106.98,132.25,-0.04,0.000000
7,CDUAF,0.95,0.44,-0.22,1.34,24.15,24.44,0.01,0.72,0.52,4.18,4.70,26.12,0.08,0.62,0.64,3.78,12.92,22.20,-0.08,0.000000
8,COTY,0.89,0.61,3.14,3.29,6.81,7.25,0.06,0.64,0.50,-1.17,4.55,8.06,0.18,0.62,0.62,3.13,-0.55,6.70,-0.02,0.000000
9,COVTY,1.92,0.53,9.12,-3.99,28.65,30.17,0.05,1.08,0.52,-7.84,11.73,31.70,0.11,1.11,0.53,1.24,0.82,27.91,-0.03,0.000000
10,CDMGF,3.14,0.38,-9.33,22.99,22.23,22.97,0.03,2.19,0.21,-64.97,-15.58,23.73,0.07,1.96,0.17,-11.78,-9.40,22.89,0.03,0.000000


In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
23,VNM,0.63,0.88,50.95,80.05,11.17,12.34,0.10,0.39,0.69,22.26,60.16,12.31,0.10,0.41,0.64,27.93,71.59,11.53,0.03,7.600000
17,ILF,1.16,0.87,118.59,109.31,21.39,23.14,0.08,0.84,0.62,63.02,68.20,22.70,0.06,0.89,0.70,57.86,75.10,20.77,-0.03,7.075000
4,EPHE,1.27,0.85,107.98,124.74,24.25,25.98,0.07,0.85,0.56,11.61,39.18,26.44,0.09,0.90,0.64,48.21,84.93,24.23,-0.00,6.357143
7,EWI,1.50,0.83,165.86,132.17,36.60,33.61,-0.08,1.08,0.56,40.11,5.10,36.07,-0.01,0.94,0.58,64.02,22.61,32.72,-0.11,6.175000
15,EWZ,1.61,0.85,123.70,171.42,22.92,24.23,0.06,1.12,0.56,68.35,71.93,25.63,0.12,1.11,0.73,92.58,117.43,22.76,-0.01,6.000000
21,THD,2.77,0.77,170.04,253.84,57.22,64.46,0.13,1.90,0.53,15.19,56.10,70.36,0.23,2.04,0.65,55.82,135.17,58.26,0.02,5.930769
11,EWP,1.12,0.79,133.60,74.91,31.36,28.65,-0.09,0.89,0.72,110.54,56.40,31.25,-0.00,0.77,0.62,50.19,21.48,28.44,-0.09,5.566667
16,IDX,0.83,0.80,38.57,58.91,14.53,15.75,0.08,0.57,0.52,-4.17,19.54,16.00,0.10,0.72,0.56,7.01,27.13,14.78,0.02,5.500000
2,FLN,0.80,0.81,65.81,77.14,15.36,16.50,0.07,0.65,0.65,48.50,41.68,17.87,0.16,0.63,0.62,45.98,52.83,14.97,-0.03,5.385714
0,ARGT,3.92,0.63,398.44,3.14,87.44,72.62,-0.17,2.61,0.55,227.88,-58.55,73.87,-0.16,2.36,0.39,85.28,-261.55,64.27,-0.27,4.123529


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False

tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, True)

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
87,TEX,0.91,0.97,534.75,584.77,44.95,52.29,0.16,0.54,0.75,541.15,556.98,53.59,0.19,0.51,0.76,405.10,431.43,42.40,-0.06,9.512500
64,OLN,0.89,0.94,389.80,551.16,32.66,38.94,0.19,0.56,0.70,231.61,370.46,39.94,0.22,0.60,0.80,478.81,566.54,30.39,-0.07,9.084211
13,BGFV,0.38,0.92,47.42,144.10,1.73,2.32,0.35,0.25,0.62,23.79,97.64,2.24,0.30,0.20,0.80,66.84,141.12,1.35,-0.21,8.971429
37,F,0.23,0.95,129.00,138.72,9.71,10.53,0.08,0.13,0.74,84.26,127.94,11.91,0.23,0.10,0.74,80.41,107.42,8.74,-0.10,8.875000
66,PARA,0.77,0.90,116.67,294.79,10.65,14.91,0.40,0.41,0.61,127.69,224.77,12.28,0.15,0.46,0.79,175.44,288.91,9.43,-0.11,8.750000
3,AMR,5.58,0.91,2883.89,2144.78,189.68,237.56,0.25,3.14,0.81,3064.17,2751.07,233.85,0.23,4.15,0.70,2442.91,1069.35,191.10,0.01,8.740000
91,TWI,0.29,0.92,113.80,152.57,6.58,7.72,0.17,0.16,0.71,99.13,137.22,7.40,0.13,0.19,0.77,124.04,136.00,6.47,-0.02,8.729412
65,OSK,1.43,0.96,661.61,807.61,91.85,95.26,0.04,0.80,0.71,731.48,703.30,103.95,0.13,0.84,0.76,628.52,864.73,88.92,-0.03,8.600000
38,FCX,0.75,0.94,369.36,418.92,39.05,41.84,0.07,0.43,0.71,298.85,262.33,44.27,0.13,0.53,0.83,435.44,518.47,35.13,-0.10,8.542857
11,BAX,0.96,0.91,165.80,519.92,30.46,35.46,0.16,0.61,0.62,45.68,282.05,38.44,0.26,0.64,0.84,199.25,492.49,30.01,-0.01,8.537500


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, False)

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
15,WEAT,0.25,0.85,36.55,52.21,4.91,6.20,0.26,0.18,0.56,51.35,49.65,5.70,0.16,0.11,0.73,34.32,43.83,4.81,-0.02,7.923077
1,FXB,1.84,0.92,201.62,242.74,118.52,116.79,-0.01,1.14,0.75,228.57,186.36,120.65,0.02,2.01,0.71,165.20,237.11,115.34,-0.03,1.200000
8,SLV,1.79,0.60,72.04,42.33,27.96,25.76,-0.08,1.26,0.60,80.87,-6.68,27.60,-0.01,1.52,0.50,32.64,-17.32,25.49,-0.09,1.000000
2,FXE,1.38,0.85,130.12,179.60,96.16,97.66,0.02,0.98,0.58,16.69,58.29,97.55,0.01,1.18,0.58,44.43,73.53,94.69,-0.02,1.000000
11,TLT,4.50,0.74,123.41,416.23,87.78,84.43,-0.04,4.70,0.41,-232.61,0.00,94.90,0.08,3.75,0.70,111.09,288.32,82.13,-0.06,0.900000
4,FXY,2.44,0.62,13.47,165.16,59.38,62.15,0.05,1.36,0.41,-90.42,42.36,62.17,0.05,2.16,0.57,-46.46,103.67,59.16,-0.00,0.000000
0,DBC,1.03,0.57,41.18,44.31,22.34,21.43,-0.04,0.74,0.57,41.34,14.62,22.19,-0.01,0.75,0.51,23.57,5.73,20.95,-0.06,0.000000
3,FXF,1.45,0.67,52.97,93.34,97.87,98.34,0.00,0.98,0.54,70.95,40.36,99.16,0.01,0.92,0.70,48.11,144.68,96.05,-0.02,0.000000
7,PDBC,0.58,0.76,36.13,36.74,13.54,13.21,-0.02,0.33,0.69,48.98,49.61,13.54,-0.00,0.46,0.51,5.67,1.71,11.94,-0.12,0.000000
6,ISHG,0.94,0.88,130.12,147.17,67.06,67.27,0.00,0.80,0.59,28.68,69.41,68.31,0.02,0.87,0.76,65.89,141.37,66.73,-0.00,0.000000


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, True)

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
16,XRP-USD,0.04,0.88,22.84,11.20,3.17,1.54,-0.51,0.04,0.59,18.56,5.68,2.52,-0.21,0.03,0.63,7.98,-4.12,1.28,-0.60,6.050420
2,BTC-USD,1472.88,0.89,828623.43,477821.07,105022.42,80147.25,-0.24,1642.52,0.73,686009.31,373635.46,103679.77,-0.01,1181.56,0.56,293891.33,-119210.61,66526.41,-0.37,5.898810
6,ETH-USD,86.08,0.90,47103.70,43865.06,3308.15,3683.19,0.11,74.21,0.69,38953.14,38099.46,3634.56,0.10,63.62,0.70,22605.10,19374.57,2503.14,-0.24,5.519481
11,MATIC-USD,0.06,0.88,13.76,23.47,0.45,0.52,0.15,0.05,0.60,11.83,19.43,0.53,0.16,0.04,0.65,7.39,12.75,0.44,-0.02,5.485714
1,AVAX-USD,2.20,0.84,647.80,904.72,36.49,45.70,0.25,2.26,0.71,812.03,994.39,45.46,0.25,1.48,0.66,381.01,794.93,31.53,-0.14,5.360000
13,SOL-USD,5.24,0.84,2211.50,1783.16,244.88,184.01,-0.25,6.13,0.54,2464.67,1340.92,211.00,-0.14,4.67,0.54,983.97,-61.48,120.30,-0.51,5.360000
15,TRX-USD,0.00,0.82,1.64,0.85,0.24,0.17,-0.30,0.00,0.78,1.80,0.70,0.24,-0.01,0.00,0.44,0.94,-0.70,0.13,-0.44,5.257143
10,LTC-USD,4.77,0.82,1032.44,1390.54,121.24,156.55,0.29,5.36,0.63,910.94,825.72,142.23,0.17,3.77,0.67,852.95,1010.42,102.75,-0.15,5.236453
3,BCH-USD,19.65,0.83,6289.79,6073.08,443.90,521.74,0.18,20.13,0.55,4830.27,3664.11,543.68,0.22,11.68,0.69,3938.39,3714.63,342.29,-0.23,4.984127
14,TON-USD,0.27,0.70,32.78,50.28,0.14,0.81,4.92,0.36,0.65,73.91,100.50,0.70,4.12,0.21,0.62,15.73,32.38,0.36,1.65,4.939024


In [11]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False).head(50)


In [12]:

df = show_prediction('etf', '0202')
df

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
38,XLB,88.79,2025-01-31,2025-02-28,88.13,101.84,94.51,0.067988
10,FPX,131.42,2025-01-31,2025-02-28,130.68,147.53,139.43,0.059301
47,XLV,146.87,2025-01-31,2025-02-28,144.84,159.36,158.29,0.049658
13,IPO,46.92,2025-01-31,2025-02-28,46.14,50.67,50.26,0.044828
4,DAPP,15.18,2025-01-31,2025-02-28,14.62,17.00,15.92,0.043917
31,URA,28.31,2025-01-31,2025-02-28,27.68,31.26,29.70,0.043683
32,VNQ,90.55,2025-01-31,2025-02-28,89.14,95.40,97.48,0.038174
11,ICLN,11.37,2025-01-31,2025-02-28,11.16,12.33,11.92,0.038112
30,TDIV,80.75,2025-01-31,2025-02-28,80.02,88.33,82.90,0.037152
28,SMOG,99.15,2025-01-31,2025-02-28,96.63,106.53,105.05,0.036174


In [ ]:
# read csv file
pred_other1 = pd.read_csv('results/global_predictions_1a.csv')
# Join with df3['Ticker] by pred_other['ticker']
pred_other1 = pred_other1.merge(df5[['Ticker', 'Pred', 'Pred_h', 'Pred_l', 'Gain', 'Alloc']], left_on='ticker', right_on='Ticker', how='left')


In [ ]:

result1 = pred_other1[(pred_other1['predicted_close'] > pred_other1['current_price']) & (pred_other1['Gain'] > 0)]
# And largest based on 'Alloc
result1 = result1.sort_values('Alloc', ascending=False)
result1

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,Ticker,Pred,Pred_h,Pred_l,Gain,Alloc
61,THQQF,2.76,2025-01-03,2025-01-31,2.65,2.82,2.78,THQQF,3.45,4.28,2.93,0.25,1.388571
0,ASML,714.36,2025-01-03,2025-01-31,691.26,789.68,757.36,ASML,726.59,790.66,663.80,0.02,0.000000
2,ADYEY,14.78,2025-01-03,2025-01-31,14.16,15.47,15.05,ADYEY,15.40,16.87,14.27,0.04,0.000000
1,ADDDF,239.00,2025-01-03,2025-01-31,229.15,253.52,244.28,ADDDF,251.01,263.10,235.07,0.05,0.000000
3,BAK,3.78,2025-01-03,2025-01-31,3.66,4.16,3.99,BAK,4.45,5.14,4.00,0.18,0.000000
4,BASFY,10.66,2025-01-03,2025-01-31,10.30,11.54,11.00,BASFY,11.01,11.67,10.43,0.03,0.000000
8,COTY,6.81,2025-01-03,2025-01-31,6.58,7.51,7.21,COTY,7.25,8.06,6.70,0.06,0.000000
5,BESIY,137.69,2025-01-03,2025-01-31,132.71,148.89,143.05,BESIY,142.03,151.01,132.25,0.03,0.000000
13,DNNGY,15.97,2025-01-03,2025-01-31,15.25,16.98,16.03,DNNGY,17.36,19.44,15.36,0.09,0.000000
16,EDNMF,36.35,2025-01-03,2025-01-31,33.96,36.22,37.41,EDNMF,37.23,38.80,34.70,0.02,0.000000


In [ ]:
# Read stock_0123_20250123.csv
df = pd.read_csv('stock_0130_predictions.csv')
df.head(10)


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close
0,BA,177.90,2025-01-30,2025-02-27,173.74,182.27,175.05
1,AMD,118.51,2025-01-30,2025-02-27,115.87,130.37,127.62
2,TDC,32.63,2025-01-30,2025-02-27,32.09,34.41,34.13
3,SE,125.44,2025-01-30,2025-02-27,123.60,139.22,140.13
4,T,24.11,2025-01-30,2025-02-27,23.13,25.24,24.27
5,SFIX,4.93,2025-01-30,2025-02-27,4.81,5.31,5.15
6,IRDM,28.27,2025-01-30,2025-02-27,27.43,29.42,31.91
7,TGLS,78.17,2025-01-30,2025-02-27,72.59,80.46,78.91
8,SBLK,15.44,2025-01-30,2025-02-27,13.63,16.18,15.25
9,INTA,72.06,2025-01-30,2025-02-27,68.72,77.45,68.79


In [ ]:
# List stocks sorted by predicted_close_return descendent
# List stock sorted by ((predicted_max+predicted_min+predicted_close)/3 - current_price) descendent
df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']

df.sort_values('predicted_return', ascending=False).head(50)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
43,RKLB,28.22,2025-01-30,2025-02-27,27.00,46.03,31.08,0.229742
176,ASPN,11.79,2025-01-30,2025-02-27,11.56,14.51,14.69,0.152389
63,SEDG,13.09,2025-01-30,2025-02-27,12.29,16.80,15.68,0.140056
85,LUMN,4.95,2025-01-30,2025-02-27,4.82,5.88,5.85,0.114478
232,VIV,8.73,2025-01-30,2025-02-27,8.53,10.03,10.13,0.095456
200,BIIB,146.15,2025-01-30,2025-02-27,143.24,179.97,155.58,0.092006
16,STNE,9.52,2025-01-30,2025-02-27,9.10,12.64,9.40,0.090336
188,TSM,207.54,2025-01-30,2025-02-27,198.47,228.90,247.29,0.083582
146,HOOD,53.43,2025-01-30,2025-02-27,49.29,60.12,63.82,0.080729
119,AMR,183.63,2025-01-30,2025-02-27,179.75,202.85,211.72,0.078836


In [13]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
etf_0202_all_trades.csv
etf_0202_close_model.pth
etf_0202_max_model.pth
etf_0202_min_model.pth
etf_0202_predictions.csv
etftickers.csv

sent 10,705,135 bytes  received 130 bytes  21,410,530.00 bytes/sec
total size is 10,702,084  speedup is 1.00
